In [21]:
import os
import pandas as pd
from shutil import copytree

In [18]:
dirs = os.listdir("/projects/p30041/spn1560/hiec/artifacts/model_evals/gnn")
dirs.remove("warmup_min_ops_pred")
dirs

['sprhea_sp_folded_pt_mfp_ffn_homology_80_0',
 'sprhea_sp_folded_pt_mean_agg_w_wo_vn_0',
 'sprhea_sp_folded_pt_vn_agg_depths_homology_80_0',
 'sprhea_sp_folded_pt_simple_rxn_mean_agg_depths_homology_80_0',
 'sprhea_sp_folded_pt_mfp_linear_homology_80_0',
 'sprhea_sp_folded_pt_attention_homology_80_0']

In [31]:
# Load toc(s)


tmp = []

for dir in dirs:
    ds_toc = "sprhea_sp_folded_pt"
    gs_path = f"/projects/p30041/spn1560/hiec/artifacts/model_evals/gnn/{dir}"
    subdirs = os.listdir(gs_path)
    subdirs.remove("gs_params.txt")
    subdirs.remove("hp_toc.csv")
    gs_params_path = f"{gs_path}/gs_params.txt"
    toc_path = f"{gs_path}/hp_toc.csv"

    # Read in existing hp_toc info
    df = pd.read_csv(toc_path, sep='\t')
    df.drop(columns=['Unnamed: 0'], inplace=True)

    # Add gs_name to df
    df['gs_name'] = dir.split('_pt_')[-1]
    df['dataset'] = 'sprhea'
    df['toc'] = 'sp_folded_pt'
    
    # Add gs_params info
    with open(gs_params_path, 'r') as f:
        for line in f.readlines():
            k, v = line.strip().split(": ")
            if k in ["n_splits", 'neg_multiple', 'seed']:
                v = int(v)
            elif k in ['similarity_threshold']:
                v = float(v)

            df[k] = v
    
    
    tmp.append(df)

hp_toc = pd.concat(tmp)
hp_toc.reset_index(inplace=True)
hp_toc.rename(columns={"index": "hp_idx"}, inplace=True)
hp_toc.head(200)

,hp_idx,n_epochs,pred_head,message_passing,agg,d_h_encoder,model,featurizer,encoder_depth,gs_name,dataset,toc,n_splits,split_strategy,similarity_threshold,neg_multiple,embed_type,seed
0,0,25,dot_sig,NaN,NaN,20,ffn,mfp,3.0,mfp_ffn_homology_80_0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
1,1,25,dot_sig,NaN,NaN,20,ffn,mfp,2.0,mfp_ffn_homology_80_0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
2,2,25,dot_sig,NaN,NaN,20,ffn,mfp,1.0,mfp_ffn_homology_80_0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
3,3,25,dot_sig,NaN,NaN,50,ffn,mfp,3.0,mfp_ffn_homology_80_0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
4,4,25,dot_sig,NaN,NaN,50,ffn,mfp,2.0,mfp_ffn_homology_80_0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,1,25,binary,bondwise_dict,attention,50,mpnn_dim_red,rxn_rc,3.0,attention_homology_80_0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
74,2,25,binary,bondwise_dict,attention,300,mpnn_dim_red,rxn_rc,3.0,attention_homology_80_0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
75,3,25,dot_sig,bondwise_dict,attention,20,mpnn_dim_red,rxn_rc,3.0,attention_homology_80_0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
76,4,25,dot_sig,bondwise_dict,attention,50,mpnn_dim_red,rxn_rc,3.0,attention_homology_80_0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234


In [30]:
old_dir = "/projects/p30041/spn1560/hiec/artifacts/model_evals/gnn"
new_dir = "/projects/p30041/spn1560/hiec/artifacts/model_evals/out"

if not os.path.exists(new_dir):
    os.mkdir(new_dir)

for i, row in hp_toc.iterrows():
    for j in range(row['n_splits']):
        fn_in = f"{old_dir}/{row['dataset']}_{row['toc']}_{row['gs_name']}/{row['hp_idx']}_hp_idx_split_{j+1}_of_{row['n_splits']}"
        fn_out = f"{new_dir}/{i}_hp_idx_split_{j+1}_of_{row['n_splits']}"
        copytree(fn_in, fn_out)

In [32]:
hp_toc.drop(columns=['hp_idx', 'gs_name'], inplace=True)
hp_toc.head()

,n_epochs,pred_head,message_passing,agg,d_h_encoder,model,featurizer,encoder_depth,dataset,toc,n_splits,split_strategy,similarity_threshold,neg_multiple,embed_type,seed
0,25,dot_sig,NaN,NaN,20,ffn,mfp,3.0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
1,25,dot_sig,NaN,NaN,20,ffn,mfp,2.0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
2,25,dot_sig,NaN,NaN,20,ffn,mfp,1.0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
3,25,dot_sig,NaN,NaN,50,ffn,mfp,3.0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234
4,25,dot_sig,NaN,NaN,50,ffn,mfp,2.0,sprhea,sp_folded_pt,5,homology,0.8,1,esm,1234


In [35]:
hp_toc.to_csv(f"{new_dir}/experiments.csv", sep='\t')

In [34]:
df = pd.read_csv(toc_path, sep='\t', index_col=0)
df.head()

,n_epochs,pred_head,message_passing,agg,d_h_encoder,model,featurizer,encoder_depth
0,25,binary,bondwise_dict,attention,20,mpnn_dim_red,rxn_rc,3
1,25,binary,bondwise_dict,attention,50,mpnn_dim_red,rxn_rc,3
2,25,binary,bondwise_dict,attention,300,mpnn_dim_red,rxn_rc,3
3,25,dot_sig,bondwise_dict,attention,20,mpnn_dim_red,rxn_rc,3
4,25,dot_sig,bondwise_dict,attention,50,mpnn_dim_red,rxn_rc,3
